In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import math
import numpy as np
import plotly as pl
import plotly.graph_objects as go
import seaborn
import pandas
import matplotlib.pyplot as plt
import sys
import os


from fpdf import FPDF

C:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\fpdf\__init__.py:39: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [2]:
class CSection:
    def __init__(self, width_a, depth_b, flange_length_c, thickness_t, length_of_additional_csection_l_c):
        self.width_a = width_a
        self.depth_b = depth_b
        self.flange_length_c = flange_length_c
        self.thickness_t = thickness_t
        self.length_of_additional_csection_l_c = length_of_additional_csection_l_c
    
    def area(self):
        return ((self.width_a + 2 * self.flange_length_c) * self.thickness_t)
    
    def surface_area_per_unit_length(self):
        return (2 * self.flange_length_c + self.width_a + 2 * self.depth_b) / 1000
    
    def __str__(self):
        return f"CSection: Width A = {self.width_a}, Depth B = {self.depth_b}, Flange Length C = {self.flange_length_c}, Thickness T = {self.thickness_t}"


# --------------------------------------------------------------------------------------------------------------------------------

class CPileSectionSelector:
    
    factor_safety_set = {
                         "base": [1, 1.7, 1.5],
                         "shaft_compression": [1, 1.5, 1.3],
                         "total_compression": [1, 1.7, 1.5],
                         "shaft_tension": [1, 2, 1.7],
                         "lateral_resistance": [1, 1.5, 1.5]
                        }
    
    
    def return_correlation_factors_epsilon(self, number_of_tests, a_correlation_reduction_factor):
        a = a_correlation_reduction_factor
        if number_of_tests == 1:
            return [1.55 / a, 1.55 / a]
        elif number_of_tests == 2:
            return [1.47 / a, 1.35 / a]
        elif number_of_tests == 3:
            return [1.42 / a, 1.23 / a]
        elif number_of_tests == 4:
            return [1.38 / a, 1.15 / a]
        elif number_of_tests >= 5:
            b = 1 if 1.08 / a < 1 else 1.08 / a
            return [1.35 / a, b]
    
    
    def __init__(self, pot_data, test_type, section, a_correlation_reduction_factor, length_of_pile_Ld=2.5):  
        self.a_correlation_reduction_factor = a_correlation_reduction_factor
        self.test_type = test_type
        self.c_section_data_selector = []
        self.pot_data = pot_data
        self.section = section
        self.length_of_pile_Ld = length_of_pile_Ld
        
        
    def add_c_section(self, c_section):  
        self.c_section_data_selector.append(c_section)
        
# --------------------------------------------------------------------------------------------------------------------------------

    def run_calculations(self):
        
        if self.test_type == "shaft_tension":
            self.return_design_value_tension(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_tension()
            self.print_results_tension()
            self.print_additional_sections_tension()
            
        elif self.test_type == "lateral_resistance":
            self.return_design_value_lateral_resistance(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_lateral_resistance()
            self.print_results_lateral_resistance()
            self.print_additional_sections_lateral_resistance()

        elif self.test_type == "shaft_compression":
            self.return_design_value_shaft_compression(self.a_correlation_reduction_factor)
            self.calculate_additional_sections_shaft_compression()
            self.print_results_shaft_compression()
            self.print_additional_sections_shaft_compression()
            
# --------------------------------------------------------------------------------------------------------------------------------
            
    def return_design_value_tension(self, a_correlation_reduction_factor):
        
        correlation_factors_epsilon = self.return_correlation_factors_epsilon(len(self.pot_data), a_correlation_reduction_factor)
        
        self.R_t_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_t_m_min = min(self.pot_data)
        
        self.R_t_k_characteristic = self.R_t_m_mean / correlation_factors_epsilon[0] if self.R_t_m_mean / correlation_factors_epsilon[0] <= self.R_t_m_min / correlation_factors_epsilon[1] else self.R_t_m_min / correlation_factors_epsilon[1]
    
        self.R_t_design_R1 = self.R_t_k_characteristic / self.factor_safety_set[self.test_type][0] 
        self.R_t_design_R4_without_SLS = self.R_t_k_characteristic / self.factor_safety_set[self.test_type][1] 
        self.R_t_design_R4_with_SLS = self.R_t_k_characteristic / self.factor_safety_set[self.test_type][2] 
        
        self.skin_surface_area = self.section.surface_area_per_unit_length() * self.length_of_pile_Ld
        
        self.q_su_R1 = self.R_t_design_R1 / self.skin_surface_area
        self.q_su_R4_without_SLS = self.R_t_design_R4_without_SLS / self.skin_surface_area
        self.q_su_R4_with_SLS = self.R_t_design_R4_with_SLS / self.skin_surface_area
    
    def return_design_value_lateral_resistance(self, a_correlation_reduction_factor):
        
        correlation_factors_epsilon = self.return_correlation_factors_epsilon(len(self.pot_data), a_correlation_reduction_factor)
        
        self.R_l_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_l_m_min = min(self.pot_data)
        
        self.R_l_k_characteristic = self.R_l_m_mean / correlation_factors_epsilon[0] if self.R_l_m_mean / correlation_factors_epsilon[0] <= self.R_l_m_min / correlation_factors_epsilon[1] else self.R_l_m_min / correlation_factors_epsilon[1]
        
        self.R_l_design_R1 = self.R_l_k_characteristic / self.factor_safety_set[self.test_type][0] 
        self.R_l_design_R4_without_SLS = self.R_l_k_characteristic / self.factor_safety_set[self.test_type][1] 
        self.R_l_design_R4_with_SLS = self.R_l_k_characteristic / self.factor_safety_set[self.test_type][2] 
        
        self.q_lateral_R1 = self.R_l_design_R1 / self.section.width_a
        self.q_lateral_R4_without_SLS = self.R_l_design_R4_without_SLS / self.section.width_a
        self.q_lateral_R4_with_SLS = self.R_l_design_R4_with_SLS / self.section.width_a

    # q_cu might need revisions in the future models !!!
    def return_design_value_shaft_compression(self, a_correlation_reduction_factor):

        correlation_factors_epsilon = self.return_correlation_factors_epsilon(len(self.pot_data), a_correlation_reduction_factor)
        
        self.R_c_m_mean = sum(self.pot_data) / len(self.pot_data)
        self.R_c_m_min = min(self.pot_data)
        
        self.R_c_k_characteristic = self.R_c_m_mean / correlation_factors_epsilon[0] if self.R_c_m_mean / correlation_factors_epsilon[0] <= self.R_c_m_min / correlation_factors_epsilon[1] else self.R_c_m_min / correlation_factors_epsilon[1]
        
        self.R_c_design_R1 = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][0] 
        self.R_c_design_R4_without_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][1] 
        self.R_c_design_R4_with_SLS = self.R_c_k_characteristic / self.factor_safety_set[self.test_type][2]

        self.skin_surface_area = self.section.surface_area_per_unit_length() * self.length_of_pile_Ld
        
        self.q_cu_R1 = self.R_c_design_R1 / self.skin_surface_area
        self.q_cu_R4_without_SLS = self.R_c_design_R4_without_SLS / self.skin_surface_area
        self.q_cu_R4_with_SLS = self.R_c_design_R4_with_SLS / self.skin_surface_area

# --------------------------------------------------------------------------------------------------------------------------------

    def calculate_additional_sections_tension(self):
        
        self.additional_section_results_tension = []

        for sec in self.c_section_data_selector:
            skin_surface_area = sec.surface_area_per_unit_length() * sec.length_of_additional_csection_l_c

            R_t_R1 = self.q_su_R1 * skin_surface_area
            R_t_R4_without_SLS = self.q_su_R4_without_SLS * skin_surface_area
            R_t_R4_with_SLS = self.q_su_R4_with_SLS * skin_surface_area

            self.additional_section_results_tension.append({
                "section": sec,
                "R_t_R1": R_t_R1,
                "R_t_R4_without_SLS": R_t_R4_without_SLS,
                "R_t_R4_with_SLS": R_t_R4_with_SLS,
            })
    
    def calculate_additional_sections_lateral_resistance(self):
        
        self.additional_section_results_lateral_resistance = []

        for sec in self.c_section_data_selector:
            R_l_R1 = self.q_lateral_R1 * sec.width_a
            R_l_R4_without_SLS = self.q_lateral_R4_without_SLS * sec.width_a
            R_l_R4_with_SLS = self.q_lateral_R4_with_SLS * sec.width_a
            
            self.additional_section_results_lateral_resistance.append({
                "section": sec,
                "R_l_R1": R_l_R1,
                "R_l_R4_without_SLS": R_l_R4_without_SLS,
                "R_l_R4_with_SLS": R_l_R4_with_SLS,
            })

    def calculate_additional_sections_shaft_compression(self):
        
        self.additional_section_results_shaft_compression = []

        for sec in self.c_section_data_selector:
            skin_surface_area = sec.surface_area_per_unit_length() * sec.length_of_additional_csection_l_c

            R_c_R1 = self.q_cu_R1 * skin_surface_area
            R_c_R4_without_SLS = self.q_cu_R4_without_SLS * skin_surface_area
            R_c_R4_with_SLS = self.q_cu_R4_with_SLS * skin_surface_area

            self.additional_section_results_shaft_compression.append({
                "section": sec,
                "R_c_R1": R_c_R1,
                "R_c_R4_without_SLS": R_c_R4_without_SLS,
                "R_c_R4_with_SLS": R_c_R4_with_SLS,
            })

# --------------------------------------------------------------------------------------------------------------------------------

    def print_additional_sections_tension(self):
        
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.additional_section_results_tension, 1):
            print(f"Additional Section {index}, Pile Length: {result["section"].length_of_additional_csection_l_c}")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_t_R1 for Section {index}:", f"{result['R_t_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_t_R4_without_SLS for Section {index}:", f"{result['R_t_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_t_R4_with_SLS for Section {index}:", f"{result['R_t_R4_with_SLS']:.2f}kN"))
            print()
            
    def print_additional_sections_lateral_resistance(self):
        
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.additional_section_results_lateral_resistance, 1):
            print(f"Additional Section {index}, Pile Length: {result["section"].length_of_additional_csection_l_c}")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_l_R1 for Section {index}:", f"{result['R_l_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_l_R4_without_SLS for Section {index}:", f"{result['R_l_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_l_R4_with_SLS for Section {index}:", f"{result['R_l_R4_with_SLS']:.2f}kN"))
            print()

    def print_additional_sections_shaft_compression(self):
        
        print("ADDITIONAL SECTIONS RESULTS")
        print(f"------------------------------")
        for index, result in enumerate(self.additional_section_results_shaft_compression, 1):
            print(f"Additional Section {index}, Pile Length: {result["section"].length_of_additional_csection_l_c}")
            print(f"------------------------------")
            print("{:<80} {:>10}".format(f"R_c_R1 for Section {index}:", f"{result['R_c_R1']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_without_SLS for Section {index}:", f"{result['R_c_R4_without_SLS']:.2f}kN"))
            print("{:<80} {:>10}".format(f"R_c_R4_with_SLS for Section {index}:", f"{result['R_c_R4_with_SLS']:.2f}kN"))
            print()

# --------------------------------------------------------------------------------------------------------------------------------

    def print_results_tension(self):

        print()
        print("PILE SECTION DESIGN RESULTS FOR TENSION RESISTANCE")
        print(f"------------------------------")
        print()
        
        # Geometry
        print(f"------------------------------")
        print("Geometry of CSection")
        print(f"------------------------------")

        print()
        # POT Data and Tests
        print(f"------------------------------")
        print("POT Data and Tests")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Number of POT Tests:", len(self.pot_data)))
        print("{:<80} {:>10}".format("POT Data:", f"{self.pot_data}"))

        # Length
        print()
        print(f"------------------------------")
        print(f"Pile Length: {self.length_of_pile_Ld}")
        print(f"------------------------------")
        print()
        
        # Design Values
        print(f"------------------------------")
        print("Design Values")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("R_t_m_mean:", f"{self.R_t_m_mean:.2f}kN"))
        print("{:<80} {:>10}".format("R_t_m_min:", f"{self.R_t_m_min:.2f}kN"))
        print("{:<80} {:>10}".format("R_t_k_characteristic:", f"{self.R_t_k_characteristic:.2f}kN"))
        print("{:<80} {:>10}".format("R_t_design_R1:", f"{self.R_t_design_R1:.2f}kN"))
        print("{:<80} {:>10}".format("R_t_design_R4_without_SLS:", f"{self.R_t_design_R4_without_SLS:.2f}kN"))
        print("{:<80} {:>10}".format("R_t_design_R4_with_SLS:", f"{self.R_t_design_R4_with_SLS:.2f}kN"))
        
    def print_results_lateral_resistance(self):
        
        print()
        print("PILE SECTION DESIGN RESULTS FOR LATERAL RESISTANCE")
        print(f"------------------------------")
        print()
        
        # Geometry
        print(f"------------------------------")
        print("Geometry of CSection")
        print(f"------------------------------")

        print()
        # POT Data and Tests
        print(f"------------------------------")
        print("POT Data and Tests")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Number of POT Tests:", len(self.pot_data)))
        print("{:<80} {:>10}".format("POT Data:", f"{self.pot_data}"))

        # Length
        print()
        print(f"------------------------------")
        print(f"Pile Length: {self.length_of_pile_Ld}")
        print(f"------------------------------")
        print()
        
        # Design Values
        print(f"------------------------------")
        print("Design Values")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("R_l_m_mean:", f"{self.R_l_m_mean:.2f}kN"))
        print("{:<80} {:>10}".format("R_l_m_min:", f"{self.R_l_m_min:.2f}kN"))
        print("{:<80} {:>10}".format("R_l_k_characteristic:", f"{self.R_l_k_characteristic:.2f}kN"))
        print("{:<80} {:>10}".format("R_l_design_R1:", f"{self.R_l_design_R1:.2f}kN"))
        print("{:<80} {:>10}".format("R_l_design_R4_without_SLS:", f"{self.R_l_design_R4_without_SLS:.2f}kN"))
        print("{:<80} {:>10}".format("R_l_design_R4_with_SLS:", f"{self.R_l_design_R4_with_SLS:.2f}kN"))

    def print_results_shaft_compression(self):

        print()
        print("PILE SECTION DESIGN RESULTS FOR COMPRESSION RESISTANCE")
        print(f"------------------------------")
        print()
        
        # Geometry
        print(f"------------------------------")
        print("Geometry of CSection")
        print(f"------------------------------")

        print()
        # POT Data and Tests
        print(f"------------------------------")
        print("POT Data and Tests")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("Number of POT Tests:", len(self.pot_data)))
        print("{:<80} {:>10}".format("POT Data:", f"{self.pot_data}"))

        # Length
        print()
        print(f"------------------------------")
        print(f"Pile Length: {self.length_of_pile_Ld}")
        print(f"------------------------------")
        print()
        
        # Design Values
        print(f"------------------------------")
        print("Design Values")
        print(f"------------------------------")
        print("{:<80} {:>10}".format("R_c_m_mean:", f"{self.R_c_m_mean:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_m_min:", f"{self.R_c_m_min:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_k_characteristic:", f"{self.R_c_k_characteristic:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R1:", f"{self.R_c_design_R1:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_without_SLS:", f"{self.R_c_design_R4_without_SLS:.2f}kN"))
        print("{:<80} {:>10}".format("R_c_design_R4_with_SLS:", f"{self.R_c_design_R4_with_SLS:.2f}kN"))
    
            
# --------------------------------------------------------------------------------------------------------------------------------

    def generate_pdf_tension(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Helvetica", size=14)
        pdf.multi_cell(180, 10, f"PILE SECTION TENSION RESISTANCE RESULTS for {area}", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, "------------------------------", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Geometry
#         pdf.multi_cell(190, 10, "Geometry of CSection", align='C')

#         for index, c_section in enumerate(self.c_section_data_selector, 1):
#             pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "POT Data and Tests", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"POT Data: {self.pot_data}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Design Values
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "Design Values", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"R_t_m_mean: {self.R_t_m_mean:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_t_m_min: {self.R_t_m_min:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_t_k_characteristic: {self.R_t_k_characteristic:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        pdf.multi_cell(180, 10, f"R_t_design_R1: {self.R_t_design_R1:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_t_design_R4_without_SLS: {self.R_t_design_R4_without_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_t_design_R4_with_SLS: {self.R_t_design_R4_with_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Additional Sections
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "ADDITIONAL SECTIONS RESULTS", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        for index, result in enumerate(self.additional_section_results_tension, 1):
            c_section = self.c_section_data_selector[index - 1]

            pdf.set_font("Helvetica", size=11)
            pdf.multi_cell(180, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2,\
            CSection Pile Length = {c_section.length_of_additional_csection_l_c}m")
            
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_t_R1 for Section {index}: {result['R_t_R1']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_t_R4_without_SLS for Section {index}: {result['R_t_R4_without_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_t_R4_with_SLS for Section {index}: {result['R_t_R4_with_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(0, 10, '')
            pdf.set_xy(10, pdf.get_y())


    def generate_pdf_lateral_resistance(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Helvetica", size=14)
        pdf.multi_cell(180, 10, f"PILE SECTION LATERAL RESISTANCE RESULTS for {area}", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, "------------------------------", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Geometry
#         pdf.multi_cell(190, 10, "Geometry of CSection", align='C')
#         for index, c_section in enumerate(self.c_section_data_selector, 1):
#             pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "POT Data and Tests", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"POT Data: {self.pot_data}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Design Values
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "Design Values", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"R_l_m_mean: {self.R_l_m_mean:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_l_m_min: {self.R_l_m_min:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_l_k_characteristic: {self.R_l_k_characteristic:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        pdf.multi_cell(180, 10, f"R_l_design_R1: {self.R_l_design_R1:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_l_design_R4_without_SLS: {self.R_l_design_R4_without_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_l_design_R4_with_SLS: {self.R_l_design_R4_with_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Lateral Resistance
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "Lateral Resistance Results", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        for index, result in enumerate(self.additional_section_results_lateral_resistance, 1):
            c_section = self.c_section_data_selector[index - 1]

            pdf.set_font("Helvetica", size=11)
            pdf.multi_cell(180, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2,\
            CSection Pile Length = {c_section.length_of_additional_csection_l_c}m")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_l_R1 for Section {index}: {result['R_l_R1']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_l_R4_without_SLS for Section {index}: {result['R_l_R4_without_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_l_R4_with_SLS for Section {index}: {result['R_l_R4_with_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(0, 10, '')
            pdf.set_xy(10, pdf.get_y())

    
    def generate_pdf_shaft_compression(self, pdf, area):
        pdf.add_page()
        pdf.set_margins(left=10, top=10, right=10)
        pdf.set_font("Helvetica", size=14)
        pdf.multi_cell(180, 10, f"PILE SECTION COMPRESSION RESISTANCE RESULTS for {area}", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, "------------------------------", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Geometry
#         pdf.multi_cell(190, 10, "Geometry of CSection", align='C')
#         for index, c_section in enumerate(self.c_section_data_selector, 1):
#             pdf.multi_cell(190, 10, f"CSection {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2")

        # POT Data and Tests
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "POT Data and Tests", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"Number of POT Tests: {len(self.pot_data)}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"POT Data: {self.pot_data}")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Design Values
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "Design Values", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.set_font("Helvetica", size=11)
        pdf.multi_cell(180, 10, f"R_c_m_mean: {self.R_c_m_mean:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_c_m_min: {self.R_c_m_min:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_c_k_characteristic: {self.R_c_k_characteristic:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        pdf.multi_cell(180, 10, f"R_c_design_R1: {self.R_c_design_R1:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_c_design_R4_without_SLS: {self.R_c_design_R4_without_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(180, 10, f"R_c_design_R4_with_SLS: {self.R_c_design_R4_with_SLS:.2f}kN")
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        # Lateral Resistance
        pdf.set_font("Helvetica", size=12)
        pdf.multi_cell(180, 10, "Lateral Resistance Results", align='C')
        pdf.set_xy(10, pdf.get_y())
        pdf.multi_cell(0, 10, '')
        pdf.set_xy(10, pdf.get_y())


        for index, result in enumerate(self.additional_section_results_shaft_compression, 1):
            c_section = self.c_section_data_selector[index - 1]

            pdf.set_font("Helvetica", size=11)
            pdf.multi_cell(180, 10, f"C Section {index}: {c_section}, Surface area = {c_section.surface_area_per_unit_length()}m2,\
            CSection Pile Length = {c_section.length_of_additional_csection_l_c}m")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_c_R1 for Section {index}: {result['R_c_R1']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_c_R4_without_SLS for Section {index}: {result['R_c_R4_without_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(180, 10, f"R_c_R4_with_SLS for Section {index}: {result['R_c_R4_with_SLS']:.2f}kN")
            pdf.set_xy(10, pdf.get_y())
            pdf.multi_cell(0, 10, '')
            pdf.set_xy(10, pdf.get_y())




# --------------------------------------------------------------------------------------------------------------------------------
        


In [3]:
# THINGS TO CHANGE HERE!!!!!

section1 = CSection(250, 110, 55, 12, 1.8)
section2 = CSection(200, 100, 50, 10, 1.8)
section3 = CSection(180, 90, 45, 9, 1.8)
section4 = CSection(160, 80, 35, 7, 1.8)
section5 = CSection(140, 60, 30, 5, 1.8)
section6 = CSection(120, 50, 25, 5, 1.8)
section7 = CSection(100, 45, 25, 5, 1.8)
section8 = CSection(80, 40, 12, 3, 1.8)

data = {"ZONE 1": [15, 20, 25, 25, 25], "ZONE 2": [25, 15, 12.5, 12, 15], "ZONE 3": [25, 20, 25, 25, 15], "ZONE 4": [20, 16, 12.5, 20, 15]}
# data = {"ZONE 1": [16, 13.5, 16, 19.5, 16.5, 16, 16, 16, 16, 16, 16, 16, 16]}

data2 = {"ZONE 1": [5, 5, 5, 5, 5], "ZONE 2": [5, 5, 5, 5, 5], "ZONE 3": [5, 5, 5, 5, 7.5], "ZONE 4": [7.5, 7.5, 5, 5, 7.5]}
# data2 = {"ZONE 1": [5, 5, 5, 5, 5]}

a_correlation_reduction_factor = 1.1

pdf = FPDF()
pdf.set_auto_page_break(auto=1, margin=15)

# factor_safety_set = {
#                          "base": [1, 1.7, 1.5],
#                          "shaft_compression": [1, 1.5, 1.3],
#                          "total_compression": [1, 1.7, 1.5],
#                          "shaft_tension": [1, 2, 1.7],
#                          "lateral_resistance": [1, 1.5, 1.5]
#                         }

for area, pot_data in data.items():
    selector = CPileSectionSelector(pot_data, "shaft_tension", section7, a_correlation_reduction_factor, length_of_pile_Ld=2.0)
    
    # Add your c_sections here (You would replace these lines with your actual sections)
    selector.add_c_section(section1)
    selector.add_c_section(section2)
    selector.add_c_section(section3)
    selector.add_c_section(section4)
    selector.add_c_section(section5)
    selector.add_c_section(section6)
    selector.add_c_section(section7)
    selector.add_c_section(section8)

    selector.run_calculations()
    selector.generate_pdf_tension(pdf, area)

for area, pot_data in data2.items():
    selector = CPileSectionSelector(pot_data, "lateral_resistance", section7, a_correlation_reduction_factor, length_of_pile_Ld=1.8)
    
    # Add your c_sections here (You would replace these lines with your actual sections)
    selector.add_c_section(section1)
    selector.add_c_section(section2)
    selector.add_c_section(section3)
    selector.add_c_section(section4)
    selector.add_c_section(section5)
    selector.add_c_section(section6)
    selector.add_c_section(section7)
    selector.add_c_section(section8)

    selector.run_calculations()
    selector.generate_pdf_lateral_resistance(pdf, area)

for area, pot_data in data.items():
    selector = CPileSectionSelector(pot_data, "shaft_compression", section7, a_correlation_reduction_factor, length_of_pile_Ld=1.8)
    
    # Add your c_sections here (You would replace these lines with your actual sections)
    selector.add_c_section(section1)
    selector.add_c_section(section2)
    selector.add_c_section(section3)
    selector.add_c_section(section4)
    selector.add_c_section(section5)
    selector.add_c_section(section6)
    selector.add_c_section(section7)
    selector.add_c_section(section8)

    selector.run_calculations()
    selector.generate_pdf_shaft_compression(pdf, area)
    

pdf.output(name="DUNFERMLINE_POT_Results.pdf")


PILE SECTION DESIGN RESULTS FOR TENSION RESISTANCE
------------------------------

------------------------------
Geometry of CSection
------------------------------

------------------------------
POT Data and Tests
------------------------------
Number of POT Tests:                                                                      5
POT Data:                                                                        [15, 20, 25, 25, 25]

------------------------------
Pile Length: 2.0
------------------------------

------------------------------
Design Values
------------------------------
R_t_m_mean:                                                                         22.00kN
R_t_m_min:                                                                          15.00kN
R_t_k_characteristic:                                                               15.00kN
R_t_design_R1:                                                                      15.00kN
R_t_design_R4_without_SLS:      